# Load the user connection data and connect to the SAP HANA database instance

Before running the next cell make sure [../0x00-setup/temp_user.ini](../0x00-setup/temp_user.ini) is copied to [../0x00-setup/user.ini](../0x00-setup/user.ini) and the user + password are set appropriately.

In [ ]:
from hana_ml.algorithms.pal.utility import Settings
myhost, myport, myuser, mypwd = Settings.load_config("../0x00-setup/user.ini")

In [ ]:
from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser,
    password=mypwd
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")

In [ ]:
print(myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0])

# Tables from SAP HANA

In [ ]:
hdf_titanic_train=myconn.table('TRAIN', schema='TITANIC')

# Random Decision Tree classification

Experiment 3: Exclude high cardinality features (names, tickets and cabins) to improve generalization of the model

In [ ]:
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification

In [ ]:
uc_rdt_v3 = UnifiedClassification(func='RandomDecisionTree')

In [ ]:
features_low_cardinality=['Age', 'SibSp', 'ParCh', 'PClass', 'Fare', 'Gender', 'Embarked']
#list(set(hdf_titanic_train.columns) - set(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Survived'])) ## <--define features manually

In [ ]:
uc_rdt_v3.fit(
    data=hdf_titanic_train,
    key='PassengerId', label='Survived',
    features=features_low_cardinality, #pass only low-cardinality features
    training_percent=0.8,
    partition_method='stratified', stratified_column='Survived', 
    partition_random_state=2
);

## Generate a model report

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport
UnifiedReport(uc_rdt_v3).build().display()

## Debrief the model

In [ ]:
from hana_ml.visualizers.model_debriefing import TreeModelDebriefing

In [ ]:
TreeModelDebriefing.tree_debrief_with_dot(uc_rdt_v3.model_[0], iframe_height=700);

# Store the model

In [ ]:
from hana_ml.model_storage import ModelStorage

Model storage: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/hana_ml.model_storage.html

In [ ]:
ms = ModelStorage(myconn)

In [ ]:
uc_rdt_v3.name = 'CodeJam-Titanic-Classification - Experiement 0x030'
ms.save_model(model=uc_rdt_v3, save_report=False)

In [ ]:
ms.list_models()

In [ ]:
null = None
false = False
true = True
eval(ms.list_models(name=uc_rdt_v3.name).at[0, 'JSON'])

## Call predition

In [ ]:
hdf_titanic_test=myconn.table('TEST', schema='TITANIC')

In [ ]:
hdf_res_v3 = uc_rdt_v3.predict(hdf_titanic_test, key = 'PassengerId')

In [ ]:
display(hdf_res_v3.sort('CONFIDENCE', desc=True).head(3).collect())
display(hdf_res_v3.sort('CONFIDENCE', desc=True).tail(3).collect())

## Compare to the ground truth

In [ ]:
hdf_titanic_complete=myconn.table('COMPLETE', schema='TITANIC')

In [ ]:
hdf_res_ext=hdf_res_v3.set_index('PassengerId').join(hdf_titanic_test.set_index('PassengerId'))

In [ ]:
hdf_res_ext.head(3).collect()

In [ ]:
hdf_res_incl_groundtruth=(hdf_res_ext.set_index(['Name', 'Ticket']).join(hdf_titanic_complete.set_index(['name', 'ticket']))
                 .select('PassengerId', 'Name', 'Ticket', 'SCORE','survived',('1-ABS(SCORE-"survived")', 'IS_CORRECT'))
                 .cast('SCORE', 'TINYINT')

)

### Accuracy

In [ ]:
hdf_res_incl_groundtruth.select(('SUM("IS_CORRECT")/COUNT("IS_CORRECT")','Accuracy')).collect()

### Confusion matrix

In [ ]:
hdf_res_incl_groundtruth.agg(agg_list=[('count','PassengerId','COUNT')], group_by=['SCORE','survived']).collect()

In [ ]:
df_res_incl_gt_pivot=(hdf_res_incl_groundtruth
 .pivot_table(values='PassengerId', index='SCORE', columns='survived', aggfunc='count')
 .collect()
);

Order columns and rows by 0,1, and remove the Pandas DataFrame index

In [ ]:
(df_res_incl_gt_pivot[[df_res_incl_gt_pivot.columns[0]]+sorted(df_res_incl_gt_pivot.columns[1:])]
 .sort_values(by=df_res_incl_gt_pivot.columns[0], axis=0)
 .style.hide(axis='index')
)

🤓 **Let's discuss**:
- Comparison of the last two models you trained